In [1]:
import numpy as np
from numba import njit

In [2]:
@njit
def diffusion_iteration(u,kernel0,kernel,E,rho,t,dt,loadfunc):
    
    N, Tsteps = u.shape
    ut1 = np.zeros(N)
    ut2 = np.zeros(N)
    horizon = np.s ize(kernel)
    for tt in range(Tsteps-1):
        for i in range(horizon,N-horizon):
            temp = -kernel0*u[i,tt] # temporary variable
            
            for k in range(horizon):
                temp += kernel[k]*(u[i+k+1,tt]+u[i-k-1,tt])
            
            # use the explicit finite difference for time
            ut2[i] = E/rho*temp
            u[i,tt+1] = u[i,tt] + dt*(ut1[i]+ut2[i])/2
            ut1[i] = ut2[i]
        
            # BC at the fixed end
        for i in range(horizon):
            u[i,tt+1]= -u[2*horizon-1-i,tt+1]
        
            # BC with displacement load
        for i in range(N-horizon,N):
            P = loadfunc(t[tt])
            u[i,tt+1] = 2*P-u[2*(N-horizon)-1-i,tt+1]
            
    return


class simulator_1D():
    
    def __init__(self,E,rho,loadfunc,Ttotal,dt,Nnodes):
        self.E = E
        self.rho = rho
        self.loadfunc = loadfunc
        self.Ttotal = Ttotal
        self.dt = dt
        self.Nnodes = Nnodes
        
        
    def nonlocal_kernel_sim(self,kernel):
    
        # This is the function that use the given nonlocal kernel and the given material properties and the displacement
        # BC to do a simulation on 1-D elastic dynamic equation
        
        E = self.E
        rho = self.rho
        loadfunc = self.loadfunc
        Ttotal = self.Ttotal
        dt = self.dt
        Nnodes = self.Nnodes
    
        horizon = np.size(kernel)
        self.horizon = horizon
        kernel0 = 2*np.sum(kernel) # for symmetric kernel only
        Tsteps = int(Ttotal/dt)
        N = Nnodes+2*horizon # add some nodes at the boundaries for nonlocal BCs
        self.nonlocal_N = N
        u = np.zeros((N,Tsteps),dtype=np.double)
        t = np.linspace(0,Ttotal,Tsteps)
        self.t = t
    
        # some arrays for storing velocity and acceleration  
        
        diffusion_iteration(u,kernel0,kernel,E,rho,t,dt,loadfunc)
        
        
        ut1 = np.zeros(N)
        ut2 = np.zeros(N)
        
        


        """"
        for tt in range(Tsteps-1):
            for i in range(horizon,N-horizon):
                temp = -kernel0*u[i,tt] # temporary variable
            
                for k in range(horizon):
                    temp += kernel[k]*(u[i+k+1,tt]+u[i-k-1,tt])
            
                # use the explicit finite difference for time
                ut2[i] = E/rho*temp
                u[i,tt+1] = u[i,tt] + dt*(ut1[i]+ut2[i])/2
                ut1[i] = ut2[i]
        
            # BC at the fixed end
            for i in range(horizon):
                u[i,tt+1]= -u[2*horizon-1-i,tt+1]
        
            # BC with displacement load
            for i in range(N-horizon,N):
                P = loadfunc(t[tt])
                u[i,tt+1] = 2*P-u[2*(N-horizon)-1-i,tt+1]
        """
    
        return u
    
    def nonlocal_kernel_middisplacement(self,kernel):
        
        # This function returns only the displacement at the midpoint of the rod
        
        u = self.nonlocal_kernel_sim(kernel)
        umid = (u[int(self.nonlocal_N/2),:]+u[int(self.nonlocal_N/2-1),:])/2
        
        return umid
    
    


        
    

In [3]:
x = np.zeros((3,5))

In [4]:
np.size(x)

15

In [10]:
kernel = np.array([ 1.61341853e+02, -1.14752190e+01,  8.16159280e-01, -5.80482142e-02,
        4.12859996e-03, -2.93641034e-04])

In [11]:
np.size(kernel)
def displacement_load(t):
    T = 1570
    P0 = -50e3
    b0 = 1e-17
    #load = P0*b0
    load = P0*b0*t**6*(t-T)**6*(1-np.heaviside(t-T,0))
    return load

In [12]:
mytest = simulator_1D(1.1089e-7,6.8948e-4, displacement_load,100000,1,50)

In [13]:
mytest.nonlocal_kernel_sim(kernel)

TypingError: Failed at nopython (nopython frontend)
Internal error at <numba.typeinfer.ArgConstraint object at 0x6127f0c18>:
--%<----------------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/xiao/anaconda3/lib/python3.6/site-packages/numba/errors.py", line 491, in new_error_context
    yield
  File "/Users/xiao/anaconda3/lib/python3.6/site-packages/numba/typeinfer.py", line 194, in __call__
    assert ty.is_precise()
AssertionError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/xiao/anaconda3/lib/python3.6/site-packages/numba/typeinfer.py", line 138, in propagate
    constraint(typeinfer)
  File "/Users/xiao/anaconda3/lib/python3.6/site-packages/numba/typeinfer.py", line 195, in __call__
    typeinfer.add_type(self.dst, ty, loc=self.loc)
  File "/Users/xiao/anaconda3/lib/python3.6/contextlib.py", line 99, in __exit__
    self.gen.throw(type, value, t